# Modelo 5. Optimización Lineal

### Leer los datos

In [1]:
import pandas as pd

# ---------------
# IMPORTACIÓN DE DATOS
# ---------------
df_m = pd.read_excel('DT5.xlsx', sheet_name='Medicos')
df_s = pd.read_excel('DT5.xlsx', sheet_name='Pacientes')
df_e = pd.read_excel('DT5.xlsx', sheet_name='Clientes')

# ---------------
# CONJUNTOS
# ---------------
# # M = conjunto de médicos disponibles identificados por ID.
M = list(df_m['ID Medico'])
# S = conjunto de solicitudes identificadas por el ID del servicio.
S = list(df_s['ID Servicio'])
# E = conjunto de entidades.
E = list(df_e['Tipo'])

# ---------------
# PARÁMETROS
# ---------------
# Crear diccionario de parámetros
#------------ Parámetros asociados a los médicos
# i_m = documendo de identidad del médico m∈M.
i = dict(zip(M, df_m['Nombre Medico']))
# a_m = inicio de la jornada laboral del médico m∈M.
a = dict(zip(M, df_m['Inicio Jornada']))
# b_m = fin de la jornada laboral del médico m∈M.
b = dict(zip(M, df_m['Fin Jornada']))
# t_m = minuto en el que el médico m∈M está disponible.
t = dict(zip(M, df_m['Inicio Jornada']))
# c_m = número de solicitudes de servicio que se han  asignado al médico m∈M. 
c = dict(zip(M, [0]*len(M)))
lat = 4.681230925773865
lon = -74.06200456196707
# u_(i,j)^m = coordenadas i,j de la última ubicación geográfica del médico m∈M.
u = dict(zip(M, list(zip([lat]*len(M), [lon]*len(M)))))

#------------ Parámetros asociados a las solicitudes
# h_s = minuto en el que se recibió la solicitud s∈S.
h = dict(zip(S, df_s['Llegada']))
# d_s = duración de la atención de la solicitud s∈S.
d = dict(zip(S, df_s['Atencion']))
# e_s = entidad que solicitó el servicio s∈S.
e = dict(zip(S, df_s['Entidad']))
# v_(i,j)^s = coordenadas i,j de la ubicación geográfica de la solicitud s∈S.
v = dict(zip(S, list(zip(df_s['Latitud'], df_s['Longitud']))))

#------------ Parámetros asociados a las entidades
# f_e = tiempo en min. dentro del cual se debe atender las solicitudes de la entidad e∈E.
f = dict(zip(E, df_e['Tiempo']))

### Calcular la distancia/tiempo entre cada par de nodos

In [2]:
from math import radians, cos, sin, asin, sqrt
def distancia_tierra(lat1, lat2, lon1, lon2):
    
    # Convertir grados a radianes.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Formula Haversine, para calcular la distancia entre dos puntos de una esfera dadas sus coordenadas de longitud y latitud
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la tierra en km.
    r = 6371
    
    respuesta = ((c * r)/24.9)*60
    # Calcular la distancia final en Km.
    return respuesta

### Cambiar datos para Gantt

In [3]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

### Asignación de los médicos

In [4]:
import pulp as lp

gantt = []
for s in S:

    # Tiempo máximo en el que se puede atender la solicitud s de acuerdo al tiempo contratado de la entidad.
    t_max = h[s] + f[e[s]]

    # Diccionario de distancias.
    dist = {(m, s): 0 for m in M}
    # Calcular las distancias entre los médicos y la solicitud y guardarlas en el diccionario.
    for m in M:
        distancia = distancia_tierra(v[s][0], u[m][0], v[s][1], u[m][1])
        dist[(m, s)] = distancia

    # Diccionario de pesos para balancaer la carga laboral
    p = {m: None for m in M}

    for m in M:

        if c[m] <= 2:
            p[m] = c[m] * 0.45 + (t[m] + dist[(m,s)]) * 0.55
        elif c[m] <= 4:
            p[m] = c[m] * 0.35 + (t[m] + dist[(m,s)]) * 0.65
        else:
            p[m] = c[m] * 0.25 + (t[m] + dist[(m,s)]) * 0.75

    # ---------------
    # MODELO DE OPTIMIZACIÓN
    # ---------------
    # Crear problema de optimización.
    prob = lp.LpProblem('Agendamiento_de_medicos_ADOM', lp.LpMinimize)

    # Crear variable de decisión.
    x = lp.LpVariable.dicts('x', [(s,m) for m in M], 0, 1, lp.LpBinary)

    # FUNCIÓN OBJETIVO
    prob += lp.lpSum(p[m]*x[(s,m)] for m in M)

    # RESTRICCIONES

    # 1. No se debe exceder el tiempo contratado por las entidades.    
    for m in M:
        if (t[m] + dist[(m, s)]) > t_max:
            prob += x[(s,m)] == 0

    # 2. No se puede asignar una solicitud a un médico por fuera de su jornada laboral.
    for m in M:
        if (t[m] + dist[(m,s)]) > b[m]:
            prob += x[(s,m)] == 0

    # 3. Restringir que cada solicitud sea asignada a un único médico.
    prob += lp.lpSum(x[(s,m)] for m in M) == 1

    # Optimizar el problema.
    prob.solve()

    # ---------------
    # ACTUALIZAR PARÁMETROS
    # ---------------
    print('Solicitud: ', s)
    contador = 0
    for m in M:
        if x[(s,m)].value() == 1:
            print('Se asignó médico: ', m)
            
            c[m] = c[m] + 1
            if h[s] < t[m]:
                gantt.append({'Solicitud': s , 'Médico': m, 'Inicio': convertMin(t[m] + dist[(m, s)]), 'Fin': convertMin(t[m] + dist[(m, s)] + d[s]), 'Distancia': dist[(m, s)], 'Inicio Jornada': a[m], 'Fin Jornada': b[m], 'Inicio traslado': t[m], 'Fin traslado': t[m] + dist[(m, s)], 'Inicio atención': t[m] + dist[(m, s)], 'Fin atención': t[m] + dist[(m, s)] + d[s], 'Llegada solicitud': h[s]})
                t[m] = t[m] + dist[(m,s)] + d[s]
            else:
                gantt.append({'Solicitud': s , 'Médico': m, 'Inicio': convertMin(h[s] + dist[(m, s)]), 'Fin': convertMin(h[s] + dist[(m, s)] + d[s]), 'Distancia': dist[(m, s)], 'Inicio Jornada': a[m], 'Fin Jornada': b[m], 'Inicio traslado': t[m], 'Fin traslado': h[s] + dist[(m, s)], 'Inicio atención': h[s] + dist[(m, s)], 'Fin atención': h[s] + dist[(m, s)] + d[s], 'Llegada solicitud': h[s]})
                t[m] = h[s] + dist[(m,s)] + d[s]

            print('Tiempo del médico: ', t[m])
            u[m] = (v[s][0], v[s][1])
            
        else:
            contador = contador + 1

    if contador == len(M):
        print('Minutos de la solicitud: ', h[s])
        print('Se cancela la solicitud: ', s)
            

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/marianamb/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7f/dyxrc8wx4rn5y924zhd8473m0000gn/T/4d56e3e500204459b7319eaa4916c734-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7f/dyxrc8wx4rn5y924zhd8473m0000gn/T/4d56e3e500204459b7319eaa4916c734-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 43 COLUMNS
At line 237 RHS
At line 276 BOUNDS
At line 316 ENDATA
Problem MODEL has 38 rows, 39 columns and 76 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.81521 - 0.00 seconds
Cgl0002I 37 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 2.81521 to -1.79769e+308
Probing was tried 0 t

### Gantt general

In [5]:
import plotly.express as px
import pandas as pd
#Poner los datos de la lista en un dataframe
df1 = pd.DataFrame(gantt)
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
df1['Solicitud'] = df1['Solicitud'].astype(str)
# poner la columna médico como texto
df1['Médico'] = df1['Médico'].astype(str)

fig = px.timeline(df1, x_start="Inicio", x_end="Fin", y="Médico", color="Médico",
                  hover_name="Solicitud",
                  title="Solicitudes atendidas por médicos")
fig.update_layout(xaxis_title="Tiempo", yaxis_title="Médico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()